In [1]:
import pandas as pd
import numpy as np
import json
import ast
import gzip
# pip freeze > requirements.txt  PARA CREAR EL ARCHIVO CON TODAS LAS LIBRERIAS REQUERIDAS JUNTO CON SUS VERSIONES

C:\Users\Martin\AppData\Local\Temp\ipykernel_11888\3351562134.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Definimos las rutas de los archivos:
ruta_games='Datasets/steam_games.json.gz'
ruta_items='Datasets/user_reviews.json.gz'
ruta_items='Datasets/users_items.json.gz'

Para reducir memoria: 
Fijarse que columnas son las se usan para el modelo y evaluar los registros que poseen valores nulos en esos campos, que hacemos borramos esos registros?

### **Games**

#### Carga de datos: **OK**

##### Pruebas que no funcionaron:

In [19]:
# Cargamos games: FORMA 1: No funcionó, hay un caracter que rompe las bolas
games = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    games.append(ast.literal_eval(linea))

ValueError: malformed node or string on line 1: <ast.Name object at 0x00000198AF7E49A0>

##### Pruebas que sí funcionaron:

In [3]:
# Cargamos games: FORMA 2: Funcionó correctamente
games = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
    for linea in lineas:
        # linea = linea.replace("\"", "'") 
        # Esta ultima linea no lo vamos a usar porque el 
        # metodo json.loads no reconoce las KEYS de los diccionarios si estan encerradas entre comillas simples ''
        games.append(linea) 
games[0]

'{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}\n'

In [4]:
# Chequeamos el tamaño de los datos:
len(games)

120445

In [5]:
# Quitamos el '\n' al final de cada linea:
for i in range(0,len(games)):
    games[i]= games[i].replace("\n","")


In [6]:
# Damos un vistazo a los datos:
games[0:4]

['{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}']

In [7]:
# Transformo cada una de las cadenas a diccionario de Python:
for i in range(0,len(games)):
    games[i]= json.loads(games[i])
    

In [8]:
# Damos otro vistazo a los datos:
games[0]

{'publisher': nan,
 'genres': nan,
 'app_name': nan,
 'title': nan,
 'url': nan,
 'release_date': nan,
 'tags': nan,
 'reviews_url': nan,
 'specs': nan,
 'price': nan,
 'early_access': nan,
 'id': nan,
 'developer': nan}

In [9]:
# Transformamos a DataFrame para facilitar la manipulacion los datos:
df_games = pd.DataFrame(games)

In [10]:
df_games.tail(4)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Notar que en las columnas *'genres'*, *'tags'* y *'specs'* los valores son listas.

#### Analisis Exploratorio

### **Reviews**

#### Carga de datos: **OK**

##### Pruebas que no funcionaron


In [55]:
# reviews = pd.read_json(ruta_reviews)
# No es posible leer con el metodo 'read_json' de Pandas

In [97]:
# Cargamos reviews: No funcionó, hay problemas para que cargue los datos correctamente 
reviews = []
with gzip.open(ruta_reviews, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
    for linea in lineas:
        linea = linea.replace("\'",'\"').replace('\\"',"'")
        #linea =linea.replace("\"")
        # DEBEMOS REEMPLAZAR
        # " \\\' " POR " ' "
        # "\'" POR '"'
        reviews.append(linea) 
reviews[1]

'{"user_id": "js41637", "user_url": "http://steamcommunity.com/id/js41637", "reviews": [{"funny": "", "posted": "Posted June 24, 2014.", "last_edited": "", "item_id": "251610", "helpful": "15 of 20 people (75%) found this review helpful", "recommend": True, "review": "I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8"}, {"funny": "", "posted": "Posted September 8, 2013.", "last_edited": "", "item_id": "227300", "helpful": "0 of 1 people (0%) found this review helpful", "recom

In [32]:
# Quitamos el '\n' al final de cada linea:
for i in range(0,len(reviews)):
    reviews[i]= reviews[i].replace("\n","")

AttributeError: 'dict' object has no attribute 'replace'

In [35]:
reviews[246][280:290] # Aca esta el error, como lo corregimos?

TypeError: unhashable type: 'slice'

In [99]:
# Transformo cada una de las cadenas a diccionario de Python: forma 2 ESTA TIRA ERROR
for i in range(0,len(reviews)):
    reviews[i]= json.loads(reviews[i])



JSONDecodeError: Expecting value: line 1 column 246 (char 245)

##### Pruebas que sí funcionaron:

In [33]:
# Cargamos reviews: FORMA 1: Funcionó correctamente
reviews = []
with gzip.open(ruta_reviews, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    reviews.append(ast.literal_eval(linea))


In [34]:
# Damos un vistazo a los datos
reviews[0:4]

[{'user_id': '76561197970982479',
  'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
  'reviews': [{'funny': '',
    'posted': 'Posted November 5, 2011.',
    'last_edited': '',
    'item_id': '1250',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
   {'funny': '',
    'posted': 'Posted July 15, 2011.',
    'last_edited': '',
    'item_id': '22200',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': "It's unique and worth a playthrough."},
   {'funny': '',
    'posted': 'Posted April 21, 2011.',
    'last_edited': '',
    'item_id': '43110',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at t

In [ ]:
# Transformamos a DataFrame:
df_reviews = pd.DataFrame(reviews)

In [ ]:
# Damos un vistazo a df_reviews:
df_reviews.head(4)

#### Analisis Exploratorio

### **Items**

#### Carga de datos: **OK**

##### Pruebas que no funcionaron:

In [37]:
# Cargamos games:
items = []
with gzip.open(ruta_items, 'rt', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
    for linea in lineas:
        linea = linea.replace("\'", '\"') 
        # Esta ultima linea no lo vamos a usar porque el 
        # metodo json.loads no reconoce las KEYS de los diccionarios si estan encerradas entre comillas simples ''
        items.append(linea) 
items[0]

'{"user_id": "76561197970982479", "items_count": 277, "steam_id": "76561197970982479", "user_url": "http://steamcommunity.com/profiles/76561197970982479", "items": [{"item_id": "10", "item_name": "Counter-Strike", "playtime_forever": 6, "playtime_2weeks": 0}, {"item_id": "20", "item_name": "Team Fortress Classic", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "30", "item_name": "Day of Defeat", "playtime_forever": 7, "playtime_2weeks": 0}, {"item_id": "40", "item_name": "Deathmatch Classic", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "50", "item_name": "Half-Life: Opposing Force", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "60", "item_name": "Ricochet", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "70", "item_name": "Half-Life", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "130", "item_name": "Half-Life: Blue Shift", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "300", "item_name": "Day of Defeat: Source", 

In [38]:
# Quitamos el '\n' al final de cada linea:
for i in range(0,len(items)):
    items[i]= items[i].replace("\n","")


In [39]:
# Transformo cada una de las cadenas a diccionario de Python:
for i in range(0,len(items)):
    items[i]= json.loads(items[i])
    

JSONDecodeError: Expecting ',' delimiter: line 1 column 1394 (char 1393)

In [ ]:
prueba = items[0:10]
codecs.decode(prueba,encoding='utf-8', errors='ignore')

['{"user_id": "76561197970982479", "items_count": 277, "steam_id": "76561197970982479", "user_url": "http://steamcommunity.com/profiles/76561197970982479", "items": [{"item_id": "10", "item_name": "Counter-Strike", "playtime_forever": 6, "playtime_2weeks": 0}, {"item_id": "20", "item_name": "Team Fortress Classic", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "30", "item_name": "Day of Defeat", "playtime_forever": 7, "playtime_2weeks": 0}, {"item_id": "40", "item_name": "Deathmatch Classic", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "50", "item_name": "Half-Life: Opposing Force", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "60", "item_name": "Ricochet", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "70", "item_name": "Half-Life", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "130", "item_name": "Half-Life: Blue Shift", "playtime_forever": 0, "playtime_2weeks": 0}, {"item_id": "300", "item_name": "Day of Defeat: Source",

TypeError: decoding with 'utf-8' codec failed (TypeError: a bytes-like object is required, not 'list')

##### Pruebas que sí funcionaron:

In [40]:
# Cargamos items:
items = []
with gzip.open(ruta_items, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    items.append(ast.literal_eval(linea))

In [41]:
# Damos un vistazo a como quedaron nuestros datos:
items[0:4]

[{'user_id': '76561197970982479',
  'items_count': 277,
  'steam_id': '76561197970982479',
  'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
  'items': [{'item_id': '10',
    'item_name': 'Counter-Strike',
    'playtime_forever': 6,
    'playtime_2weeks': 0},
   {'item_id': '20',
    'item_name': 'Team Fortress Classic',
    'playtime_forever': 0,
    'playtime_2weeks': 0},
   {'item_id': '30',
    'item_name': 'Day of Defeat',
    'playtime_forever': 7,
    'playtime_2weeks': 0},
   {'item_id': '40',
    'item_name': 'Deathmatch Classic',
    'playtime_forever': 0,
    'playtime_2weeks': 0},
   {'item_id': '50',
    'item_name': 'Half-Life: Opposing Force',
    'playtime_forever': 0,
    'playtime_2weeks': 0},
   {'item_id': '60',
    'item_name': 'Ricochet',
    'playtime_forever': 0,
    'playtime_2weeks': 0},
   {'item_id': '70',
    'item_name': 'Half-Life',
    'playtime_forever': 0,
    'playtime_2weeks': 0},
   {'item_id': '130',
    'item_name': 'Half-Life:

In [42]:
# Transformamos a DataFrame:
df_items = pd.DataFrame(items)

In [43]:
# Damos un vistazo a nuestro DataFrame:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


#### Analisis Exploratorio